In [10]:
###############################################
# Rebrov reactor batch script
# Ting-Chen Lee and Chris Blais
# Northeastern University
# runs through all reactor conditions
###############################################

import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
import re
import itertools
import logging
from collections import defaultdict
import git
import json

from rmgpy.molecule import Molecule
from rmgpy.data.base import Database

def save_pictures(git_path="", species_path="", overwrite=False):
    """
    Save a folder full of molecule pictures, needed for the pretty dot files.

    Saves them in the results directory, in a subfolder "species_pictures".
    Unless you set overwrite=True, it'll leave alone files that are
    already there.
    """
    dictionary_filename = git_path + "/base/chemkin/species_dictionary.txt"
    specs = Database().get_species(dictionary_filename, resonance=False)

    images_dir = os.path.join(species_path)
    os.makedirs(images_dir, exist_ok=True)
    for name, species in specs.items():
        filepath = os.path.join(images_dir, name + ".png")
        if not overwrite and os.path.exists(filepath):
            continue
        species.molecule[0].draw(filepath)

def prettydot(species_path, dotfilepath, strip_line_labels=False):
    """
    Make a prettier version of the dot file (flux diagram)

    Assumes the species pictures are stored in a directory
    called 'species_pictures' alongside the dot file.
    """
    
    pictures_directory = f"{species_path}/"

    if strip_line_labels:
        print("stripping edge (line) labels")

    reSize = re.compile('size="5,6"\;page="5,6"')
    reNode = re.compile(
        '(?P<node>s\d+)\ \[\ fontname="Helvetica",\ label="(?P<label>[^"]*)"\]\;'
    )

    rePicture = re.compile("(?P<smiles>.+?)\((?P<id>\d+)\)\.png")
    reLabel = re.compile("(?P<name>.+?)\((?P<id>\d+)\)$")

    species_pictures = dict()
    for picturefile in os.listdir(pictures_directory):
        match = rePicture.match(picturefile)
        if match:
            species_pictures[match.group("id")] = picturefile
        else:
            pass
            # print(picturefile, "didn't look like a picture")

    filepath = dotfilepath

    if not open(filepath).readline().startswith("digraph"):
        raise ValueError("{0} - not a digraph".format(filepath))

    infile = open(filepath)
    prettypath = filepath.replace(".dot", "", 1) + "-pretty.dot"
    outfile = open(prettypath, "w")

    for line in infile:
        (line, changed_size) = reSize.subn('size="12,12";page="12,12"', line)
        match = reNode.search(line)
        if match:
            label = match.group("label")
            idmatch = reLabel.match(label)
            if idmatch:
                idnumber = idmatch.group("id")
                if idnumber in species_pictures:
                    line = (
                        f'%s [ image="{pictures_directory}%s" label="" width="0.5" height="0.5" imagescale=false fixedsize=false color="none" ];\n'
                        % (match.group("node"), species_pictures[idnumber])
                    )

        # rankdir="LR" to make graph go left>right instead of top>bottom
        if strip_line_labels:
            line = re.sub('label\s*=\s*"\s*[\d.]+"', 'label=""', line)

        # change colours
        line = re.sub('color="0.7,\ (.*?),\ 0.9"', r'color="1.0, \1, 0.7*\1"', line)

        outfile.write(line)

    outfile.close()
    infile.close()
#     print(f"Graph saved to: {prettypath}")
    os.system(f'dot {prettypath} -Tpng -o{prettypath.replace(".dot", "", 1) + ".png"} -Gdpi=200')
    return prettypath

def show_flux_diagrams(self, suffix="", embed=False):
    """
    Shows the flux diagrams in the notebook.
    Loads them from disk.
    Does not embed them, to keep the .ipynb file small,
    unless embed=True. Use embed=True if you might over-write the files,
    eg. you want to show flux at different points.
    """
    import IPython

    for element in "CHON":
        for phase_object in (self.gas, self.surf):
            phase = phase_object.name
            img_file = (
                f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            )
            display(IPython.display.HTML(f"<hr><h2>{element} {phase}</h2>"))
            if embed:
                display(IPython.display.Image(filename=img_file, width=400, embed=True))
            else:
                display(IPython.display.Image(url=img_file, width=400, embed=False))

        # Now do the combined
        img_file = f"reaction_path_mass{'_' if suffix else ''}{suffix}.png"
        display(IPython.display.HTML(f"<hr><h2>Combined mass</h2>"))
        if embed:
            display(IPython.display.Image(filename=img_file, width=400, embed=True))
        else:
            display(IPython.display.Image(url=img_file, width=400, embed=False))

def save_flux_diagrams(*phases, suffix="", timepoint="", species_path=""):
    """
    Saves the flux diagrams. The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    """
    for element in "CHON":
        for phase_object in phases:
            phase = phase_object.name

            diagram = ct.ReactionPathDiagram(phase_object, element)
            diagram.title = f"Reaction path diagram following {element} in {phase}"
            diagram.label_threshold = 0.001

            dot_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.dot"
            img_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.png"
            dot_bin_path = (
                "/Users/lee.ting/Code/anaconda3/pkgs/graphviz-2.40.1-hefbbd9a_2/bin/dot" 
                #maybe try "/home/lee.ting/.conda/pkgs/graphviz-2.40.1-h21bd128_2/bin/dot"
            )
            img_path = os.path.join(os.getcwd(), img_file)
            diagram.write_dot(dot_file)

            #also make a prettydot file
            prettydot(species_path, dot_file, strip_line_labels=False)

            # print(diagram.get_data())

#             print(
#                 f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'."
#             )
            os.system(f"dot {dot_file} -Tpng -o{img_file} -Gdpi=200")
#             print(f"Wrote graphviz output file to '{img_path}'.")






In [2]:
# #atol_rtol_array = [1e-9, 1e-10,1e-11,1e-12, 1e-13,1e-14,1e-15,1e-16,1e-17,
# #                   1e-18,1e-19,1e-20,1e-21,1e-22,1e-23,1e-24,1e-25,1e-26,1e-27]
# #rtol_diff = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7,1e-8, 1e-9, 1e-10]
# atol_rtol_array = np.logspace(-29, -9, num=21)
# rtol_diff = np.logspace(1, 10, num=10)


In [13]:
#######################################################################
# Input Parameters for combustor
#######################################################################

# filepath for writing files
notebook_dir = '/work/westgroup/lee.ting/cantera/ammonia_analysis/Troubleshooting/'
git_repo = "/work/westgroup/lee.ting/cantera/ammonia/"
cti_file = git_repo + "base/cantera/chem_annotated.cti"

temp = 700# kelvin
pressure = 1 * ct.one_atm  # Pascals
volume_flow = 5.8333e-5 # [m^3/s]
#3500 Ncm3/min = 3500/e6/60 m3/s = 5.8333e-5

# NH3/O2 = 0.068
X_o2 = 0.88
x_O2_str = str(X_o2)[0:3].replace(".", "_")
X_nh3 = 0.066  
x_NH3_str = str(X_nh3)[0:8].replace(".", "_")
X_he = 1 - X_o2 - X_nh3

# sensitivity settings
sensitivity = False
sensatol = 1e-6
sensrtol = 1e-6

# Specify idealGasReactor and isothermal
reactor_type=1
energy="off"

# 14 aluminum plates, each of them containing seven semi-cylindrical microchannels of 280 µm width 
# and 140 µm depth, 9 mm long, arranged at equal distances of 280 µm 

# get git commit hash and message
rmg_model_path = "../../ammonia"
repo = git.Repo(rmg_model_path)
date = time.localtime(repo.head.commit.committed_date)
git_date = f"{date[0]}_{date[1]}_{date[2]}_{date[3]}{date[4]}"
git_sha = str(repo.head.commit)[0:6]
git_msg = str(repo.head.commit.message)[0:50].replace(" ", "_").replace("'", "_").replace("\n", "")
git_file_string = f"{git_date}_{git_sha}_{git_msg}"

# set sensitivity string for file path name
if sensitivity:
    sensitivity_str = "on"
else: 
    sensitivity_str = "off"


# constants
pi = math.pi

surf_temp = temp

mw_nh3 = 17.0306e-3  # [kg/mol]
mw_o2 = 31.999e-3  # [kg/mol]
mw_he = 4.002602e-3  # [kg/mol]

o2_ratio = X_nh3 / X_o2

# O2/NH3/He: typical is
concentrations_rmg = {"O2(2)": X_o2, "NH3(6)": X_nh3, "He": X_he}

# initialize cantera gas and surface
gas = ct.Solution(cti_file, "gas")
surf = ct.Interface(cti_file, "surface1", [gas])

# initialize temperatures 
gas.TPX = temp, pressure, concentrations_rmg
surf.TP = temp, pressure # change this to surf_temp when we want a different starting temperature for the surface

# if a mistake is made with the input, 
# cantera will normalize the mole fractions. 
# make sure that we are reporting/using 
# the normalized values
X_o2 = float(gas["O2(2)"].X)
X_nh3 = float(gas["NH3(6)"].X)
X_he = float(gas["He"].X)

# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume
number_of_reactors = 1001
rradius = 1.4e-4 #140µm to 0.00014m
rtotal_length = 9e-3 #9mm to 0.009m
rtotal_vol = 49*(rradius**2)*pi*rtotal_length# 49 channels =>2.7154e-8

rlength = rtotal_length/(number_of_reactors-1)

# divide totareactor total volume 
rvol = (rtotal_vol )/number_of_reactors

# Catalyst Surface Area
site_density = (surf.site_density * 1000)  # [mol/m^2] cantera uses kmol/m^2, convert to mol/m^2
Pt_radius = 1.15e-9 #m, 23Å/2
Pt_mass = 5.43e-4 #g
Pt_density = 2.145e7 #g/m3
Pt_total_vol = Pt_mass/Pt_density
Pt_vol = 4/3*pi*Pt_radius**3
numbers_of_Pt = Pt_total_vol/Pt_vol
cat_area_total = numbers_of_Pt*4*pi*Pt_radius**2# [m^3] 
cat_area = cat_area_total / number_of_reactors

# reactor initialization
if reactor_type == 0:
    r = ct.Reactor(gas, energy=energy)
    reactor_type_str = "Reactor"
elif reactor_type == 1:
    r = ct.IdealGasReactor(gas, energy=energy)
    reactor_type_str = "IdealGasReactor"
elif reactor_type == 2:
    r = ct.ConstPressureReactor(gas, energy=energy)
    reactor_type_str = "ConstPressureReactor"
elif reactor_type == 3:
    r = ct.IdealGasConstPressureReactor(gas, energy=energy)
    reactor_type_str = "IdealGasConstPressureReactor"

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X(H):0.99999, X(N):0.00001"

# flow controllers 
one_atm = ct.one_atm
FC_temp = 298.15

molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (X_nh3 * mw_nh3 + X_o2 * mw_o2 + X_he * mw_he)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

# set relative and absolute tolerances on the simulation
sim.rtol = 1e-11
sim.atol = 1e-22

#surf.advance_coverages(100)

#################################################
# Run single reactor
#################################################

# round numbers for filepath strings so they're easier to read
cat_area_str = "%s" % "%.3g" % cat_area

# if it doesn't already exist, create

results_path = (
    notebook_dir 
    + f"/results"
)
# create species folder for results if they don't exist already

try:
    os.makedirs(results_path, exist_ok=True)
except OSError as error:
    print(error)

gas_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.species_names]

# surface ROP reports gas and surface ROP. these values are not redundant
gas_surf_ROP_str = [i + " surface ROP [kmol/m^2 s]" for i in gas.species_names]
surf_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.species_names]

# gasrxn_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.reaction_equations()]
surfrxn_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.reaction_equations()]

output_filename = (
    results_path
    + f"/Spinning_basket_results.csv"
)

outfile = open(output_filename, "w")
writer = csv.writer(outfile)

# Sensitivity atol, rtol, and strings for gas and surface reactions if selected
# slows down script by a lot
if sensitivity:
    sim.rtol_sensitivity = sensrtol
    sim.atol_sensitivity = sensatol
    sens_species = ["NH3(6)", "O2(2)", "N2(4)", "NO(5)", "N2O(7)"]  #change THIS to your species, can add "," and other species

    # turn on sensitive reactions
    for i in range(gas.n_reactions):
        r.add_sensitivity_reaction(i)

    for i in range(surf.n_reactions):
        rsurf.add_sensitivity_reaction(i)
    
    # create column names for sensitivity data
    for j in sens_species:
        gasrxn_sens_str = [
            j + " sensitivity to " + i for i in gas.reaction_equations()
        ]
        surfrxn_sens_str = [
            j + " sensitivity to " + i for i in surf.reaction_equations()
        ]
        sens_list = gasrxn_sens_str + surfrxn_sens_str  # + gastherm_sens_str

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial", 
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
#         + gasrxn_ROP_str
        + surfrxn_ROP_str
        + sens_list
    )

else:

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial",
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
#         + gasrxn_ROP_str
        + surfrxn_ROP_str
    )

# initialize time and iteration count, set time step
t = 0.0
dt = 0.1
iter_ct = 0

# run the simulation
first_run = True
distance_mm = 0

for n in range(number_of_reactors):


    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY
    inlet.syncState()
    sim.reinitialize()
    previous_coverages = surf.coverages  # in case we want to retry
    
    # Add a first row in the CSV with just the feed
    writer.writerow(
        [
            distance_mm,
            temp,
            gas.P,
            volume_flow,
            X_nh3,
            X_o2,
            X_he,
            o2_ratio,
            gas.T,
            sim.rtol,
            sim.atol,
            reactor_type_str,
            energy,
        ]
        + list(gas.X)
        + list(surf.X)
        + list(gas.net_production_rates)
        + list(surf.net_production_rates)
#         + list(gas.net_rates_of_progress)
        + list(surf.net_rates_of_progress)
    )
    if n > 0:  
        try:
            sim.advance_to_steady_state(return_residuals=True)
        except ct.CanteraError:
            t = sim.time
            sim.set_initial_time(0)
            gas.TDY = TDY
            surf.coverages = previous_coverages
            r.syncState()
            sim.reinitialize()
            new_target_time = 0.01 * t
            logging.warning(
                f"Couldn't reach {t:.1g} s so going to try {new_target_time:.1g} s"
            )
            try:
                sim.advance(new_target_time)
            except ct.CanteraError:
                outfile.close()
                raise

    if sensitivity:
        # get sensitivity for sensitive species i (e.g. methanol) in reaction j
        for i in sens_species:
            g_nrxn = gas.n_reactions
            s_nrxn = surf.n_reactions

            gas_sensitivities = [sim.sensitivity(i, j) for j in range(g_nrxn)]
            surf_sensitivities = [
                sim.sensitivity(i, j) for j in range(g_nrxn, g_nrxn + s_nrxn)
            ]

            sensitivities_all = (
                gas_sensitivities
                + surf_sensitivities
            )

        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
#             + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
            + sensitivities_all,
        )

    else:
        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
#             + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
        )


    iter_ct += 1
    distance_mm = n * rlength * 1.0e3  # distance in mm

outfile.close()

#print(f"passed! atol={atol_rtol_array}")




H2OX <=> H2O + X

In [24]:
# reaction_eq = surf.reaction_equation()
# print(reaction_eq)
# surf.set_multiplier(0,1)
# print(surf.multiplier(1))

In [20]:
type(surf.forward_rate_constants)

numpy.ndarray

In [14]:
zippy = zip(surf.species_names, surf.X)
for i,j in zippy:
    print(i,j)


NOX 4.266060279499047e-19
OX 1.4340116821644335e-12
NX 2.2293638995407118e-08
OHX 3.361429597324686e-14
NH3X 0.000946728198603679
X(N) 0.9990532308759895
NH2_X(8) 4.783402815251601e-10
NH_X(9) 1.6990719158668296e-08
H2O_X(11) 6.7701652917492e-22
X(H) 1.1612406664140646e-09


['NOX',
 'OX',
 'NX',
 'OHX',
 'NH3X',
 'X(N)',
 'NH2_X(8)',
 'NH_X(9)',
 'H2O_X(11)',
 'X(H)']

In [21]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2(4)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

2 NX <=> N2(4) + 2 X(1) rate constant:  3.24e+09
N2O(7) + X(1) <=> N2(4) + OX rate constant:  3.40e+01


In [35]:
for i,j in enumerate(surf.reaction_equations()):
    if "NO(5)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

NO_X(14) <=> NO(5) + X(1) rate constant:  6.25e+00


In [36]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2O(7)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

NO_X(14) + N_X(13) <=> N2O(7) + 2 X(1) rate constant:  3.42e+09
N2O(7) + X(1) <=> N2(4) + O_X(9) rate constant:  6.80e+01
N2O_X(51) <=> N2O(7) + X(1) rate constant:  2.80e+16
N2O(7) + X(1) <=> N2O_X(48) rate constant:  1.07e+10


In [8]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.forward_rate_constants[i]
    
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}

{'NH2_X(8) + OX <=> NH_X(9) + OHX': 4.03e+20,
 'NH_X(9) + OHX <=> H2O_X(11) + NX': 1.4889913527758058e+17,
 'NH_X(9) + OX <=> NX + OHX': 1.7378615835034354e+16,
 'O2(2) + 2 X(H) <=> 2 OX': 6490724818691223.0,
 'H2O_X(11) + OX <=> 2 OHX': 470690284109260.0,
 '2 NX <=> N2(4) + 2 X(N)': 179867396167.96237,
 'NX + OX <=> NOX + X(H)': 141952564005.81445,
 'NH3X + OX <=> NH2_X(8) + OHX': 131948361379.28954,
 'NOX + NX <=> N2O(7) + 2 X(N)': 42106846307.5233,
 'H2O_X(11) <=> H2O(3) + X(H)': 9835901986.323668,
 'NH3(6) + X(N) <=> NH3X': 7488705381.22063,
 'N2O(7) + X(H) <=> N2(4) + OX': 383.78215654287663,
 'NOX <=> NO(5) + X(N)': 320.2226772623461}

In [9]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.reverse_rate_constants[i]
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}

{'H2O_X(11) + OX <=> 2 OHX': 1.62085883605505e+23,
 'NH2_X(8) + OX <=> NH_X(9) + OHX': 4.8370205259951106e+20,
 'NH_X(9) + OX <=> NX + OHX': 4.3993254451565536e+17,
 'NH3X + OX <=> NH2_X(8) + OHX': 2.1491371213484972e+16,
 'NOX + NX <=> N2O(7) + 2 X(N)': 498152041269641.0,
 'NOX <=> NO(5) + X(N)': 24942952982.24813,
 'NH_X(9) + OHX <=> H2O_X(11) + NX': 10945941534.870459,
 'NX + OX <=> NOX + X(H)': 10635140516.89356,
 'NH3(6) + X(N) <=> NH3X': 9080262646.706255,
 'H2O_X(11) <=> H2O(3) + X(H)': 335211354.98319596,
 'O2(2) + 2 X(H) <=> 2 OX': 144.0922050043466,
 '2 NX <=> N2(4) + 2 X(N)': 0.015482090900346248,
 'N2O(7) + X(H) <=> N2(4) + OX': 3.726943028722597e-14}

In [10]:
surf_equilib = {}
for i,j in enumerate(surf.reaction_equations()):
    if "X(H)" in j:
        surf_equilib[j]=surf.equilibrium_constants[i]
{k: v for k, v in sorted(surf_equilib.items(), key=lambda item: item[1], reverse=True)}


{'N2O(7) + X(H) <=> N2(4) + OX': 1.0297505316962606e+16,
 'O2(2) + 2 X(H) <=> 2 OX': 45045634623298.516,
 'H2O_X(11) <=> H2O(3) + X(H)': 29.342389033380858}

In [12]:
surf_equilib = {}
for i,j in enumerate(surf.reaction_equations()):
    if "X(N)" in j:
        surf_equilib[j]=surf.equilibrium_constants[i]
{k: v for k, v in sorted(surf_equilib.items(), key=lambda item: item[1], reverse=True)}

{'2 NX <=> N2(4) + 2 X(N)': 11617771612743.758,
 'NX + OX <=> NOX + X(N)': 13.347502440642662,
 'NH3(6) + X(N) <=> NH3X': 0.8247234328554469,
 'NOX + NX <=> N2O(7) + 2 X(N)': 8.452609408205074e-05,
 'NOX <=> NO(5) + X(N)': 1.283820233675813e-08}

In [42]:
sim.component_name(14)


'IdealGasReactor_4: X(1)'

In [45]:
import pandas as pd

In [7]:
# pd.read_csv('/work/westgroup/lee.ting/cantera/ammonia_analysis/results/O2_results/2021_6_15_1323_df9fd9_master_500_1000K_O88N066/IdealGasReactor/energy_off/sensitivity_off/598/results/Spinning_basket_area_1.11e-07_energy_off_temp_598_O2_0_15379310344827588_NH3_0.csv')